This version is derived from the first version, removing unnecessary parts to make the process clearer.

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import librosa
from pydub import AudioSegment
import pydub
import matplotlib.pyplot as plt
import random
from typing import List
import joblib

import torch.nn as nn
import torch
import torch.nn.functional as F


import lightning as L

from torch.utils.data import TensorDataset,DataLoader
import torchmetrics
from lightning.pytorch.callbacks import ModelCheckpoint,EarlyStopping,Callback


from lightning.pytorch.loggers import CSVLogger


from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
print(torch.__version__)
print("Is CUDA available: ", torch.cuda.is_available())
print("Is Metal available: ", torch.backends.mps.is_available())

2.4.0.dev20240420
Is CUDA available:  False
Is Metal available:  True


In [5]:
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(device)

mps


In [3]:
# if torch.backends.mps.is_available():
#     device='mps'

In [3]:
# parent_directory='../../data/train_audio'

# sub_folders=glob.glob(os.path.join(parent_directory,'*'))

# files=[]

# for folder_path in sub_folders:
#     first_file=glob.glob(os.path.join(folder_path,'*.ogg'))
#     files.extend(first_file)

In [4]:
# files

In [5]:
parent_directory='../../data/train_audio'

sub_folder_names=['purswa3','spepic1','pabflo1']


files=[]

for i in sub_folder_names:

    sub_folders=glob.glob(os.path.join(parent_directory,i))

    for folder_path in sub_folders:
        first_file=glob.glob(os.path.join(folder_path,'*.ogg'))
        files.extend(first_file)

In [6]:
files

['../../data/train_audio/purswa3/XC857310.ogg',
 '../../data/train_audio/purswa3/XC857306.ogg',
 '../../data/train_audio/purswa3/XC579300.ogg',
 '../../data/train_audio/purswa3/XC857307.ogg',
 '../../data/train_audio/purswa3/XC822001.ogg',
 '../../data/train_audio/purswa3/XC778689.ogg',
 '../../data/train_audio/purswa3/XC722949.ogg',
 '../../data/train_audio/purswa3/XC778659.ogg',
 '../../data/train_audio/purswa3/XC604209.ogg',
 '../../data/train_audio/purswa3/XC784018.ogg',
 '../../data/train_audio/purswa3/XC429769.ogg',
 '../../data/train_audio/purswa3/XC684691.ogg',
 '../../data/train_audio/purswa3/XC382827.ogg',
 '../../data/train_audio/purswa3/XC278753.ogg',
 '../../data/train_audio/purswa3/XC775452.ogg',
 '../../data/train_audio/purswa3/XC458036.ogg',
 '../../data/train_audio/purswa3/XC129174.ogg',
 '../../data/train_audio/purswa3/XC715602.ogg',
 '../../data/train_audio/purswa3/XC382828.ogg',
 '../../data/train_audio/purswa3/XC284531.ogg',
 '../../data/train_audio/purswa3/XC58619

In [7]:
i=0
random_files=[]
while i<20:
    idx=random.randint(0,len(files)-1)
    random_files.append(files[idx])
    i+=1


print(random_files)

remain_files = [file for file in files if file not in random_files]

files=remain_files
print(len(files))

['../../data/train_audio/purswa3/XC857308.ogg', '../../data/train_audio/purswa3/XC196261.ogg', '../../data/train_audio/spepic1/XC787691.ogg', '../../data/train_audio/purswa3/XC579300.ogg', '../../data/train_audio/purswa3/XC857308.ogg', '../../data/train_audio/purswa3/XC382827.ogg', '../../data/train_audio/purswa3/XC784018.ogg', '../../data/train_audio/purswa3/XC382828.ogg', '../../data/train_audio/spepic1/XC804336.ogg', '../../data/train_audio/purswa3/XC579300.ogg', '../../data/train_audio/purswa3/XC278753.ogg', '../../data/train_audio/pabflo1/XC164024.ogg', '../../data/train_audio/purswa3/XC715602.ogg', '../../data/train_audio/pabflo1/XC741594.ogg', '../../data/train_audio/spepic1/XC706693.ogg', '../../data/train_audio/pabflo1/XC652774.ogg', '../../data/train_audio/pabflo1/XC665955.ogg', '../../data/train_audio/pabflo1/XC632645.ogg', '../../data/train_audio/spepic1/XC768458.ogg', '../../data/train_audio/spepic1/XC305330.ogg']
113


In [8]:
# # split train and val paths
# # i decide to randomly choose one single file from each category folder as val path, and the left part as train paths

# parent_directory='../../data/train'

# sub_folders=glob.glob(os.path.join(parent_directory,'*'))


# # Used to store randomly selected file paths
# random_files = []
# # Path to store remaining files
# files=[] 

# # Loop through each subfolder
# for folder in sub_folders:
#     # Get all file paths in subfolders
#     all_files = glob.glob(os.path.join(folder, '*'))
#     if all_files:  # Make sure the folder is not empty
#         # Random select a file from the file list
#         chosen_file = random.choice(all_files)
#         # Add to random file list
#         random_files.append(chosen_file)
#         # Add the remaining files to another list
#         files.extend([file for file in all_files if file != chosen_file])

# print("Randomly selected files:", random_files)
# print("Remaining files:", files)

## data preprocessing

In [9]:
def read_audio(path:str)->pydub.audio_segment.AudioSegment:
    """
    read ogg file as pydub.audio_segment.AudioSegment for the following steps

    parametere:
        path: *.ogg file path

    return
        audio: the readed audio data
    """
    audio = AudioSegment.from_file(path, format="ogg")

    return audio    

In [10]:
# Regarding the data of a single audio, some audio information needs to be paid attention to, such as audio duration, sampling rate, bit rate and number of channels.

def audio_info(audio:pydub.audio_segment.AudioSegment):
    """
    Grab all information of the input audio

    Parameters:
        Audio: the readed audio data

    Return:
        the information of the audio
    """
    # the audio duration time (seconds)
    duration_seconds=len(audio)/1000.0

    # the audio sampling rate
    sr=audio.frame_rate

    # the num of channels
    num_channels=audio.channels

    #bit rate
    bit_rate=audio.sample_width * 8

    return duration_seconds, sr, num_channels, bit_rate



In [11]:
# Convert audio data into array

def audio2array(audio_slices:list)->np.array:
    """
    transform audio segments to arrays
    """
    audio_arrays=np.array([np.array(audio_slice.get_array_of_samples()) for audio_slice in audio_slices])

    return audio_arrays

    

In [12]:
def slice_audio_5_align(audio:pydub.audio_segment.AudioSegment)->List[pydub.audio_segment.AudioSegment]:
    """
    Slice the complete audio into multiple 5 seconds length,
    keep all slice have the same length, especially for the last slice

    Return the list of all audio segments
    """
    # set up the segment duration
    # Set up the segment duration
    segment_duration = 5 * 1000  # 5 seconds in milliseconds

    # Check if the audio is less than 5 seconds
    if len(audio) < segment_duration:
        # Calculate the required padding length
        padding_length = segment_duration - len(audio)
        # Create a silent audio segment for padding
        silence = AudioSegment.silent(duration=padding_length)
        # Pad the audio with silence
        padded_audio = audio + silence
        return [padded_audio]  # Return the padded audio as a single segment

    # If the audio is 5 seconds or longer, proceed as normal
    segments = [audio[i:i + segment_duration] for i in range(0, len(audio), segment_duration)]

    # Ensure the last segment is exactly 5 seconds long
    if len(segments[-1]) != segment_duration:
        last_segment = audio[-segment_duration:]  # Get the last 5 seconds of the audio
        segments[-1] = last_segment  # Replace the last segment with a full 5-second segment

    return segments

In [13]:
def audio_bitrate_norm(bit_rate:float,audio_array:np.array):
    """
    because the .ogg file readed through pydub would based off the audio original bit rate,
    we want the value of the audio keep small, 
    so do normalization based off the bit rate.

    Parameters:
        bit_rate: the bit rate of the audio
        audio_array: the data in array form for each single slice
    """
    audio_array_norm = audio_array / float(2**(bit_rate-1))

    return audio_array_norm
    

In [14]:
def audio_random_sampling_2(total_samples:int,audio_segment:pydub.audio_segment.AudioSegment)-> List[pydub.audio_segment.AudioSegment]:
    """
    Randomly extract audio clips and combine them into 2 seconds of audio

    Parameters:
        total_samples: the number of randomly synthesized audio clips
        audio_segment: the single audio segment in form `pydub.audio_segment.AudioSegment`

    Return:
        The list of all random extract audio clips in form `pydub.audio_segment.AudioSegment`

    """
    random_clips=[]
    clip_num=0

    # Our goal is to randomly extract a total of 2 seconds of audio
    total_duration_ms = 2*1000

    while clip_num<total_samples:
        #Store the extracted fragments
        extracted_segments = AudioSegment.silent(duration=0)  # Create a silent segment for subsequent splicing

        # Continue looping when the total length of the extracted segments is less than 2 seconds
        while extracted_segments.duration_seconds < 2:
            # Random choose a starting point
            start_ms = random.randint(0, len(audio_segment) - 1)
            # Calculate the maximum duration that can be extracted
            max_extract_ms = total_duration_ms - int(extracted_segments.duration_seconds * 1000)
            # Random determine the duration of this draw
            extract_duration_ms = random.randint(1, max_extract_ms)
            # Random extracted fragments
            extract = audio_segment[start_ms:start_ms+extract_duration_ms]
            # concat the extracted fragments
            extracted_segments += extract

        random_two_seconds = extracted_segments

        random_clips.append(random_two_seconds)

        clip_num+=1

    return random_clips

In [15]:
def timeD2preqD(normalized_clip:np.array,sr:int):
    """
    transform audio from time domain to frequency domain

    Parameters:
        normalized_clip: the single clip in array format (data)
        sr: sampling rate

    Return:
        Due to symmetry, only half the spectrum is needed.
        And becuase the range of frequencies depends on the sampling rate of the audio signal.
        we do not need the frequency info, all of them are the same.
    """
    fft = np.fft.fft(normalized_clip)
    magnitude = np.abs(fft)
    frequency = np.linspace(0, sr, len(magnitude))

    half_len = len(magnitude) // 2 
    frequency=frequency[:half_len]
    magnitude=magnitude[:half_len]

    return magnitude,frequency

In [16]:
labels_list=[]
all_audios_magnitude=[]
for path in files:
    print(path)
    labels=[]
    # extract label
    label=path.split('/')[-2]

    # read audio
    audio=read_audio(path)

    # grab audio information
    duration_secconds,sr,num_channels,bit_rate=audio_info(audio)

    # slice audio into multi 5 seconds
    slice_5_all=slice_audio_5_align(audio)

    for single_slice in slice_5_all:

        # slice audio on each 5 sec long audio and generate multi 2 sec clips
        random_clips=audio_random_sampling_2(total_samples=5,audio_segment=single_slice)

        # convert all 2 sec clips to array format
        audio_arrays_2sec=audio2array(random_clips)

        # normalize each 2 sec audio array 
        arrays_2sec_norm=[]
        for i in audio_arrays_2sec:
            array_2sec_norm=audio_bitrate_norm(bit_rate=bit_rate,audio_array=i)
            arrays_2sec_norm.append(array_2sec_norm)
        
        # arrays_2sec_norm=np.array(arrays_2sec_norm)

        # conver audio from time domain to frequency domain
        audios_magnitude=[]
        for i in arrays_2sec_norm:
            magnitude,frequency=timeD2preqD(normalized_clip=i,sr=sr)
            audios_magnitude.append(magnitude)

        # audios_magnitude=np.array(audios_magnitude)

        labels.append(label)
        all_audios_magnitude.append(audios_magnitude)


    # all_audios_magnitude=np.array(all_audios_magnitude)

    
    labels_list.append(labels)



../../data/train_audio/purswa3/XC857310.ogg
../../data/train_audio/purswa3/XC857306.ogg
../../data/train_audio/purswa3/XC857307.ogg
../../data/train_audio/purswa3/XC822001.ogg
../../data/train_audio/purswa3/XC778689.ogg
../../data/train_audio/purswa3/XC722949.ogg
../../data/train_audio/purswa3/XC778659.ogg
../../data/train_audio/purswa3/XC604209.ogg
../../data/train_audio/purswa3/XC429769.ogg
../../data/train_audio/purswa3/XC684691.ogg
../../data/train_audio/purswa3/XC775452.ogg
../../data/train_audio/purswa3/XC458036.ogg
../../data/train_audio/purswa3/XC129174.ogg
../../data/train_audio/purswa3/XC284531.ogg
../../data/train_audio/purswa3/XC586194.ogg
../../data/train_audio/purswa3/XC605218.ogg
../../data/train_audio/purswa3/XC858113.ogg
../../data/train_audio/purswa3/XC548990.ogg
../../data/train_audio/purswa3/XC573342.ogg
../../data/train_audio/purswa3/XC283188.ogg
../../data/train_audio/purswa3/XC537106.ogg
../../data/train_audio/purswa3/XC196259.ogg
../../data/train_audio/purswa3/X

In [17]:
# same step for val-set prepare

val_labels_list=[]
val_all_audios_magnitude=[]
for path in random_files:
    print(path)
    labels=[]
    # extract label
    label=path.split('/')[-2]

    # read audio
    audio=read_audio(path)

    # grab audio information
    duration_secconds,sr,num_channels,bit_rate=audio_info(audio)

    # slice audio into multi 5 seconds
    slice_5_all=slice_audio_5_align(audio)

    for single_slice in slice_5_all:

        # slice audio on each 5 sec long audio and generate multi 2 sec clips
        random_clips=audio_random_sampling_2(total_samples=5,audio_segment=single_slice)

        # convert all 2 sec clips to array format
        audio_arrays_2sec=audio2array(random_clips)

        # normalize each 2 sec audio array 
        arrays_2sec_norm=[]
        for i in audio_arrays_2sec:
            array_2sec_norm=audio_bitrate_norm(bit_rate=bit_rate,audio_array=i)
            arrays_2sec_norm.append(array_2sec_norm)
        
        # arrays_2sec_norm=np.array(arrays_2sec_norm)

        # conver audio from time domain to frequency domain
        audios_magnitude=[]
        for i in arrays_2sec_norm:
            magnitude,frequency=timeD2preqD(normalized_clip=i,sr=sr)
            audios_magnitude.append(magnitude)

        # audios_magnitude=np.array(audios_magnitude)

        labels.append(label)
        val_all_audios_magnitude.append(audios_magnitude)


    # all_audios_magnitude=np.array(all_audios_magnitude)

    
    val_labels_list.append(labels)

../../data/train_audio/purswa3/XC857308.ogg
../../data/train_audio/purswa3/XC196261.ogg
../../data/train_audio/spepic1/XC787691.ogg
../../data/train_audio/purswa3/XC579300.ogg
../../data/train_audio/purswa3/XC857308.ogg
../../data/train_audio/purswa3/XC382827.ogg
../../data/train_audio/purswa3/XC784018.ogg
../../data/train_audio/purswa3/XC382828.ogg
../../data/train_audio/spepic1/XC804336.ogg
../../data/train_audio/purswa3/XC579300.ogg
../../data/train_audio/purswa3/XC278753.ogg
../../data/train_audio/pabflo1/XC164024.ogg
../../data/train_audio/purswa3/XC715602.ogg
../../data/train_audio/pabflo1/XC741594.ogg
../../data/train_audio/spepic1/XC706693.ogg
../../data/train_audio/pabflo1/XC652774.ogg
../../data/train_audio/pabflo1/XC665955.ogg
../../data/train_audio/pabflo1/XC632645.ogg
../../data/train_audio/spepic1/XC768458.ogg
../../data/train_audio/spepic1/XC305330.ogg


In [18]:
labels_flatten_list = [element for sublist in labels_list for element in sublist]

In [19]:
all_audios_magnitude=np.array(all_audios_magnitude)

In [20]:
val_labels_flatten_list = [element for sublist in val_labels_list for element in sublist]

In [21]:
val_all_audios_magnitude=np.array(val_all_audios_magnitude)

## label encoder

In [22]:
# initialize label encoder

encoder=LabelEncoder()

# use Labelencoder to transform labels
encoded_labels=encoder.fit_transform(labels_flatten_list)

In [23]:
# If needed, you can view the mapping of original labels to encodings
label_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
label_mapping

{'pabflo1': 0, 'purswa3': 1, 'spepic1': 2}

In [24]:
# then use the loaded encoder to encode val set labels

val_encoded_labels=encoder.transform(val_labels_flatten_list)

## global normalization

In [25]:
scaler=StandardScaler()
all_audios_magnitude_norm=scaler.fit_transform(all_audios_magnitude.reshape(-1,all_audios_magnitude.shape[-1])).reshape(all_audios_magnitude.shape)

In [26]:
del all_audios_magnitude

In [27]:
val_all_audios_magnitude_norm = scaler.transform(val_all_audios_magnitude.reshape(-1, val_all_audios_magnitude.shape[-1])).reshape(val_all_audios_magnitude.shape)

In [28]:
del val_all_audios_magnitude

## convert array to tensor

In [29]:
train_features = torch.Tensor(all_audios_magnitude_norm)
val_features = torch.Tensor(val_all_audios_magnitude_norm)
train_labels = torch.Tensor(encoded_labels)
val_labels = torch.Tensor(val_encoded_labels)

In [42]:
train_features.shape

torch.Size([779, 5, 32000])

## build up neural network

In [30]:
class ConvBlock1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv1d(in_channels=5,out_channels=32,kernel_size=2,stride=2,padding=0)
        self.conv2=nn.Conv1d(in_channels=5,out_channels=32,kernel_size=4,stride=2,padding=1)
        self.conv3=nn.Conv1d(in_channels=5,out_channels=32,kernel_size=8,stride=2,padding=3)
    def forward(self,x):
        x1=self.conv1(x)
        x2=self.conv2(x)
        x3=self.conv3(x)
        # The length of the input data shape is 32000, and the stride is 2, so after conv1d, the length becomes 16000
        # The number of output channels of each conv1d layer is 32, so for the shape of the entire output, regardless of batchsize, it is 32*16000
        # Because of the chrononet architecture, we need to connect the outputs of the three layers to become 96*16000 output data.
        x=torch.cat((x1,x2,x3),dim=1)

        return x

In [31]:
class ConvBlock2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv1d(in_channels=96,out_channels=32,kernel_size=2,stride=2,padding=0)
        self.conv2=nn.Conv1d(in_channels=96,out_channels=32,kernel_size=4,stride=2,padding=1)
        self.conv3=nn.Conv1d(in_channels=96,out_channels=32,kernel_size=8,stride=2,padding=3)

    def forward(self,x):
        x1=self.conv1(x)
        x2=self.conv2(x)
        x3=self.conv3(x)
        # From the output of ConvBlock1, we know that the input shape of convBlock2 is 96*16000
        # After the calculation of this block, the output will become 96*8000
        x=torch.cat((x1,x2,x3),dim=1)

        return x

In [32]:
class ConvBlock3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv1d(in_channels=96,out_channels=32,kernel_size=2,stride=2,padding=0)
        self.conv2=nn.Conv1d(in_channels=96,out_channels=32,kernel_size=4,stride=2,padding=1)
        self.conv3=nn.Conv1d(in_channels=96,out_channels=32,kernel_size=8,stride=2,padding=3)

    def forward(self,x):
        x1=self.conv1(x)
        x2=self.conv2(x)
        x3=self.conv3(x)
        # From the output of ConvBlock1, we know that the input shape of convBlock2 is 96*8000
        # After the calculation of this block, the output will become 96*4000
        x=torch.cat((x1,x2,x3),dim=1)

        return x

In [33]:
# Because we need to train the model first, we need to calculate the loss
# For multi-classification problems, if you choose to use nn.crossentropylss, you need to remove F.softmax(),
# Because this loss function combines Log-Softmax and NLL Loss (Negative Log Likelihood Loss).

class ChronoNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1=ConvBlock1()
        self.block2=ConvBlock2()
        self.block3=ConvBlock3()
        self.bn1 = nn.BatchNorm1d(num_features=4000)
        self.gru1=nn.GRU(input_size=96,hidden_size=32,num_layers=1,batch_first=True)
        self.bn2 = nn.BatchNorm1d(num_features=4000)
        self.gru2=nn.GRU(input_size=32,hidden_size=32,num_layers=1,batch_first=True)
        self.bn3 = nn.BatchNorm1d(num_features=4000)
        self.gru3=nn.GRU(input_size=64,hidden_size=32,num_layers=1,batch_first=True)
        self.bn4 = nn.BatchNorm1d(num_features=4000)
        self.gru4=nn.GRU(input_size=96,hidden_size=32,num_layers=1,batch_first=True)
        self.bn5 = nn.BatchNorm1d(num_features=4000)
        self.fc1=nn.Linear(in_features=32,out_features=64)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, 3)  # num_classes is the number of categories



    def forward(self,x):
        x=self.block1(x)
        x=self.block2(x)
        x=self.block3(x)
        # Because the input shape required by gru is (batch_size, sequence length, feature_size)
        # But the result of the previous conversion calculation is (batchsize, feature_size, sequence length)
        # I need to change the shape
        x=x.permute(0,2,1)

        # add batch normalization
        x=self.bn1(x)
        # add relu activation fucntion
        x = F.relu(x) 

        gru_out1,_=self.gru1(x)

        x=self.bn2(gru_out1)
        x=F.relu(x)

        gru_out2,_=self.gru2(gru_out1)

        x=self.bn3(gru_out2)
        x=F.relu(x)

        # According to the chrononet architecture, we need to connect the calculations of the two layers of GRU according to the feature-size dimension
        x=torch.cat((gru_out1,gru_out2),dim=2)
        gru_out3,_=self.gru3(x)

        x=self.bn4(gru_out3)
        x=F.relu(x)

        x=torch.cat((gru_out1,gru_out2,gru_out3),dim=2)
        gru_out4,_=self.gru4(x)

        x=self.bn5(gru_out4)
        x=F.relu(x)

        x = self.fc1(gru_out4[:, -1, :]) 
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [34]:
class ChronoNetModule(L.LightningModule):
    def __init__(self,model,learning_rate):
        super().__init__()
        self.model=model
        self.lr=learning_rate
        self.train_acc=torchmetrics.Accuracy(task='multiclass',num_classes=3)
        self.val_acc=torchmetrics.Accuracy(task='multiclass',num_classes=3)



    def forward(self,x):
        '''
        x: feature data for training 

        This is the part of the neural model that is used to read or build
        define the computation performed at every call define the computation performed at every call

        return:
            model's output
        '''
        return self.model(x)
    
    def training_step(self,batch,batch_idx):
        '''
        we need to train the model right here
        including provide the loss step, acc calculation step

        This function will perform the following operations:
        1. Calculate the loss value for each training batch
        2. Perform optimization and gradient descent (automatically performed by lightningModule)
        3. Update parameters (automatically performed by lightningModule)
        https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#training
        '''
        # read batch data
        features,labels=batch

        # send data to GPU for training 
        features=features.to(self.device)
        labels=labels.to(self.device)
        
        # feeding feature to the model
        # Only self() is used here because the forward() function is called automatically
        # forward propagation
        out=self(features)

        # After getting the output of the model, you need to calculate the loss function
        loss=F.cross_entropy(out, labels)

        # logs metrics for each training_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        # After adding self.train_acc=torchmetrics.Accuracy(task='multiclass',num_classes=6)
        # You can add the steps to calculate accuracy below
        # Because we use cross_entropy() as the loss function
        # So we need to use argmax to convert to normal values ​​for accuracy calculation
        # predicted_labels=torch.argmax(out)
        # But torchmetrics.Accuracy is already configured to handle logits for multi-class classification problems. 
        # It will apply softmax (or log_softmax) and calculate argmax internally to determine the most likely category.
        acc=self.train_acc(out,labels)
        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        

        # In training_step(), we only calculate and return the loss. 
        # The optimization part does not belong to this part, and the optimization method will be defined in configure_optimizers.
        return loss # this is passed to the optimizer for training
    
    def validation_step(self,batch,batch_idx):
        '''
        val step is not used in traning, only in validation
        '''
        features,labels=batch

        # send data to GPU for training
        features=features.to(self.device)
        labels=labels.to(self.device)
        
        out=self(features)
        loss=F.cross_entropy(out, labels)

        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        acc=self.val_acc(out,labels)
        self.log("val_acc", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)


    def configure_optimizers(self):
        '''
        Choose what optimizers and learning-rate schedulers to use in your optimization.

        The optimizer defined here will be automatically called by lightningModule
        Used in the training step
        '''
        optimizer=torch.optim.Adam(self.parameters(), lr=self.lr)

        return optimizer


    def on_train_epoch_end(self):
        pass


    def on_validation_epoch_end(self):
        pass


    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        # If you have only one tensor (feature) in your TensorDataset, batch will be a tuple containing a tensor and an empty tuple (since there are no labels)
        features= batch[0]
        features=features.to(self.device)
        predictions = self(features)
        # Because what our model ultimately wants is the probability of an object corresponding to all categories, so add the softmax function here
        probabilities = torch.softmax(predictions, dim=1)

        return probabilities




In [35]:
class ChronoNetDataModule(L.LightningDataModule):
    def __init__(self,train=None,train_label=None,val=None,val_label=None,pred=None,batch_size:int=32,num_workers:int=12):
        super().__init__()
        self.batch_size=batch_size
        self.num_workers=num_workers
        self.train=train
        self.train_label=train_label
        self.val=val
        self.val_label=val_label
        self.pred=pred


    def train_dataloader(self):
        # You need to create a tensor to include the data and labels together
        # The length of the first dimension of the two must be equal
        dataset=TensorDataset(self.train,self.train_label)
        loader= DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_workers, persistent_workers=True, shuffle=True)

        return loader
    
    def val_dataloader(self):
        dataset=TensorDataset(self.val,self.val_label)
        loader= DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_workers, persistent_workers=True,shuffle=False)

        return loader

    def predict_dataloader(self):
        dataset=TensorDataset(self.pred)
        loader=DataLoader(dataset,batch_size=self.batch_size,shuffle=False)

        return loader


In [36]:
# Previous we used a separate dataloader to feed the model
# Here we encapsulate the dataloader and use this class to read data for training

dm=ChronoNetDataModule(train=train_features,train_label=train_labels,val=val_features,val_label=val_labels,batch_size=32).to(device)
print(dm)




model=ChronoNet()
ChronoNetModule=ChronoNetModule(model=model,learning_rate=0.001)

trainer=L.Trainer(
    max_epochs=3,
    accelerator="gpu", # set to 'auto' or 'gpu' to use gpu if possible
    devices=1, # use all gpus if applicable like value=1 or "auto"
    default_root_dir='/Users/yiding/personal_projects/ML/github_repo/birdcief/code/model-training/',
    logger=CSVLogger(save_dir='/Users/yiding/personal_projects/ML/github_repo/birdcief/code/model-training/log/',name='chrononet')
)

# train the model
trainer.fit(
    model=ChronoNetModule,
    datamodule=dm
)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Missing logger folder: /Users/yiding/personal_projects/ML/github_repo/birdcief/code/model-training/log/chrononet

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | ChronoNet          | 171 K 
1 | train_acc | MulticlassAccuracy | 0     
2 | val_acc   | MulticlassAccuracy | 0     
-------------------------------------------------
171 K     Trainable params
0         Non-trainable params
171 K     Total params
0.686     Total estimated model params size (MB)


/opt/homebrew/Caskroom/miniforge/base/envs/birdclef/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 25/25 [12:25<00:00,  0.03it/s, v_num=0, train_loss_step=0.937, train_acc_step=0.636, val_loss_step=0.656, val_acc_step=0.818, val_loss_epoch=1.170, val_acc_epoch=0.299, train_loss_epoch=1.040, train_acc_epoch=0.506]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 25/25 [12:25<00:00,  0.03it/s, v_num=0, train_loss_step=0.937, train_acc_step=0.636, val_loss_step=0.656, val_acc_step=0.818, val_loss_epoch=1.170, val_acc_epoch=0.299, train_loss_epoch=1.040, train_acc_epoch=0.506]


In [37]:
dm=ChronoNetDataModule(pred=val_features,batch_size=33)

predictions=trainer.predict(model=ChronoNetModule,datamodule=dm)

/opt/homebrew/Caskroom/miniforge/base/envs/birdclef/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 4/4 [00:39<00:00,  0.10it/s]


In [40]:
result = torch.cat((predictions[0], predictions[1],predictions[2],predictions[3]), dim=0)
print(result)

tensor([[0.4965, 0.2688, 0.2347],
        [0.3273, 0.2583, 0.4144],
        [0.5006, 0.2680, 0.2314],
        [0.4999, 0.2684, 0.2317],
        [0.4985, 0.2676, 0.2339],
        [0.5009, 0.2680, 0.2311],
        [0.3749, 0.2693, 0.3558],
        [0.5011, 0.2682, 0.2307],
        [0.5013, 0.2683, 0.2305],
        [0.4955, 0.2667, 0.2378],
        [0.5013, 0.2682, 0.2304],
        [0.4986, 0.2675, 0.2339],
        [0.4995, 0.2677, 0.2328],
        [0.4964, 0.2676, 0.2360],
        [0.5010, 0.2683, 0.2307],
        [0.4113, 0.2699, 0.3188],
        [0.1827, 0.2095, 0.6078],
        [0.1832, 0.2101, 0.6067],
        [0.1985, 0.2183, 0.5832],
        [0.2141, 0.2260, 0.5600],
        [0.3266, 0.2601, 0.4133],
        [0.1779, 0.2066, 0.6154],
        [0.3698, 0.2665, 0.3637],
        [0.3503, 0.2632, 0.3865],
        [0.5013, 0.2683, 0.2305],
        [0.5006, 0.2683, 0.2311],
        [0.3738, 0.2686, 0.3576],
        [0.4989, 0.2675, 0.2336],
        [0.5004, 0.2679, 0.2316],
        [0.466

In [45]:
a=torch.argmax(result,dim=1)

In [46]:
a

tensor([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
        0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0])

In [47]:
val_labels

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])